<a href="https://colab.research.google.com/github/abhik-99/MFSGC/blob/master/(Train)Multi_Filter_Supervised_Gene_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(Train)Multi-filtering Supervised Gene Clustering.

##Current Status:-
Implementing Signal to Noise Ratio and Finalizing Details.
_

**Please Run the Dataset Splitter before running this Notebook.** Provide the 
*DATASET_train.csv* generated from the Dataset Splitter Notebook as the 
input to this Notebook at cell 4 as an upload.

In [ ]:
!pip install -U -q PyDrive
!pip install skfeature-chappers

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut

from scipy.sparse import *

In [ ]:
#Uncomment the line below if you have already used the dataset splitter else
#use the original dataset
#files.upload()

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1rO5EEvsoRJl2VVUB3ywKUd3kNiQ24oy3'}) # replace the id with id of file you want to access
# For Leukemia- 1xcL-LT-E_gUqWLlqqeVJP1DVHHpiAGe_
# For Colon - 1AUOto0GhTHW9fX52XSsf9kzYJS5ggv0G
# for Prostate - 13Hf7uGbyJ1sWYo8KDRDL8scm-2Fs9_gd
# For Lung- 1xuLzTWDGUbr4x3Pq1dnJj08MZqBB5I3U
# for Rahc - 1oaOATE0D_f8MGPIMJOMXYVt0hBUWNCKV
# for Raoa - 1d2vhPcT3I7ZFcAGOQYVLGB3Jx_vEMata
# for Rbreast - 1Vf-h8zfVP_twMXivcJJtbWtjThShUHvn
# for SRBCT - 1rO5EEvsoRJl2VVUB3ywKUd3kNiQ24oy3
# for MLL - 1rS7x4x_DhrUzaBhrgKMQH3uIaLJdPgW3
# for Breast - 1enhhyA4u2ByvOjnF81WoHflVNpXtfKpu
downloaded.GetContentFile('data.txt')

In [ ]:
#DATASET is the name of the dataset being used.
DATASET="RAHC"

#NEIGHBOURS determines neighbours arg for ReliefF
#for any dataset which contains any class sample 
# <10, make it less than 10. Eg of such dataset - SRBCT
NEIGHBOURS = 3 

#p is the number of top genes taken after sorting the filter scores
p=800

#uncomment the line below if using the dataset splitter else leave it commented 
#data_df = pd.read_csv("%s_train.csv"%(DATASET),index_col=0)

#uncomment the lines below if using the original dataset
dataset = pd.read_table("data.txt",header=None)
data_df = dataset



target = data_df.iloc[:,-1]
feature = pd.DataFrame(data_df.iloc[:,:-1].values,dtype='float')
m,n = feature.shape
print(m,n)
print(feature.head())
print("Number of classes - ")
classes = np.unique(target)
for x in classes:
  print("Class -",x,"Number of Sampples -", len(np.where(target == x)[0]))

feature_norm=pd.DataFrame(MinMaxScaler().fit_transform(feature))

63 2308
     0       1       2       3     ...    2304    2305    2306    2307
0  3.2025  0.0681  1.0460  0.1243  ...  0.1521  0.3175  0.7240  0.2044
1  1.6547  0.0710  1.0409  0.0520  ...  0.1932  0.4140  1.2708  0.2990
2  3.2779  0.1160  0.8926  0.1014  ...  0.2156  0.3227  1.2142  0.2230
3  1.0060  0.1906  0.4302  0.1035  ...  0.2758  0.3016  0.7235  0.0871
4  2.7098  0.2367  0.3693  0.2190  ...  0.6412  0.3552  1.3928  0.2157

[5 rows x 2308 columns]
Number of classes - 
Class - 0.0 Number of Sampples - 23
Class - 1.0 Number of Sampples - 8
Class - 2.0 Number of Sampples - 12
Class - 3.0 Number of Sampples - 20


In [ ]:
#utility function
def plot_feature(feature, target, c = ['r', 'b', 'g', 'y']):
  import matplotlib.pyplot as plt
  from matplotlib import style
  import numpy as np
  style.use('ggplot')
  for idx, each in enumerate(np.unique(target)):
    y = feature[np.where(target == each)[0]]
    x = len(y)
    plt.scatter(range(1, x+1), y, color = c[idx])
    plt.plot(range(1, x+1), y, color = c[idx])

In [ ]:
#construction of ReliefF function

"""
Given a dataset, number of random instances to pick form the dataset and
number of features to consider in each iteration (k), the function returns the weigths of the attributes
of the dataset.
These weigths can then be used as the final results out of the ReliefF algorithm

Paper-

Marko Robnik-ˇSikonja and Igor Kononenko. Theoretical and empirical analysis of relieff
and rrelieff. Machine learning, 53(1-2):23–69, 2003.

"""

def hit_miss_calculator(target,instance,k = 10, hit = True, c = None, ):
    m=len(target)
    upper,lower=instance-1,instance+1
    hits=[]
    hit_flag=False
    #finds k nearest hits
    while(not hit_flag):
      #print(upper,lower)
      if(len(hits)>=k):
        hit_flag = True
        break
      if upper < 0 and lower > m:
        hit_flag = True
        break
      if(upper>=0):
        if((target[upper]==target[instance]) and hit):
          hits.append(upper)
        elif((target[upper]!=target[instance]) and (not hit) and target[upper]==c):
          hits.append(upper)
        upper-=1          
      if(lower<m):
        if((target[lower]==target[instance]) and hit):
          hits.append(lower)
        elif((target[lower]!=target[instance]) and (not hit) and target[lower]==c):
          hits.append(lower)
        lower+=1
    hits.sort()
    return hits


def reliefF(feature,target,k=10,repetitions=10, seed = 0):
  np.random.seed(seed)
  if len(feature.shape)>1:
    m,n=feature.shape
  else:
    m=len(feature)
    n=1
  #print(m,n)
  observations=list(range(m))
  classes=np.unique(target)
  weights=np.zeros(n)
  d=(np.max(feature,axis=0)-np.min(feature,axis=0))*m*k

  for i in range(repetitions):
    instance=np.random.choice(observations,1)[0]
    #print("Iteration",i)
    #print(instance)
    hits=hit_miss_calculator(target,instance,k)
    hit_class_prob=len(np.where(target==target[instance])[0])/m
    #print("\nHit Probability -",hit_class_prob)
    #print("Repetition",i,"Class",target[instance],"Hits -",hits)

    miss={}
    miss_class_prob={}

    for each_class in classes:
      if(each_class != target[instance]):
        miss[each_class]=hit_miss_calculator(target,instance,k,False,each_class)
        class_prob=len(np.where(target==each_class)[0])/m
        #print(each_class,class_prob)
        miss_class_prob[each_class]=hit_class_prob/(1 - (class_prob))

    #print("Repetition",i,"Miss-",miss,"Miss Class Probability -",miss_class_prob)
    
    for hit in hits:
      if len(feature.shape)>1:
        weights-=np.subtract(feature.iloc[instance,:],feature.iloc[hit,:])/d
      else:
        weights-=np.subtract(feature.iloc[instance],feature.iloc[hit])/d
    for each_class in miss:
      for each_miss in miss[each_class]:
        if len(feature.shape)>1:
          weights+=(np.subtract(feature.iloc[instance,:],feature.iloc[each_miss,:])/d)*miss_class_prob[each_class]
        else:
          weights+=(np.subtract(feature.iloc[instance],feature.iloc[each_miss])/d)*miss_class_prob[each_class]
    
    
  return weights.tolist()

In [ ]:
#This function discretizes the given features into 3 categories
def discretize_feature(feature):
  
  mean=np.mean(feature)
  std=np.std(feature)
  discretized=np.copy(feature)
  
  discretized[np.where(feature<(mean+std/2)) ,]=2#within 1/2 std div
  discretized[np.where(feature>(mean-std/2)),]=2#within 1/2 std div
  
  discretized[np.where(feature>(mean+std/2)),]=0#greater than half
  discretized[np.where(feature<(mean-std/2)),]=1#less than half
  
  return discretized

def Xfreq(x):
  xL={}
  for e in x:
    if e not in xL:
      xL[e]=0
    else:
      xL[e]+=1
  for e in xL:
    xL[e]/=len(x)
  return xL

def XYfreq(x,y):
  freq={}
  
  rX=np.unique(x)
  rY=np.unique(y)
      
  for e in rX:
    for f in rY:
      freq[(e,f)]=round(len(np.where(y[np.where(x==e)[0]]==f)[0])/len(x),4)
       
  return freq

def mutual_info(x,y):

  xFreq=Xfreq(x)
  yFreq=Xfreq(y)
  joint=XYfreq(x,y)
  
  Xentropy=0
  for e in xFreq:
    if xFreq[e]!=0:
      Xentropy-=xFreq[e]*np.log2(xFreq[e])
      
  Yentropy=0
  for e in yFreq:
    if yFreq[e]!=0:
      Yentropy-=yFreq[e]*np.log2(yFreq[e])
      
  jentropy=0
  for e in xFreq:
    for f in yFreq:
      if joint[(e,f)]!=0:
        jentropy-=joint[(e,f)]*np.log2(joint[(e,f)])
  
  return (Xentropy+Yentropy-jentropy)

def mutual_info_wrapper(features,target):

  mi=np.array([])
  for x in features:
    discrete=discretize_feature(features[x])
    mi=np.append(mi,mutual_info(discrete,target))
  return np.array(mi)

In [ ]:
"""
This cell is used for defining the method for calculating the t-scores
"""

def t_test(df,target):
  """
  Input:
  df= Dataframe of features (n_samples,n_features)
  target= Pandas Series/1D Numpy Array containing the class labels (n_samples)
  
  Output:
  scores= Descendingly Sorted array of features based on t-test 
  """
  import numpy as np
  from scipy.stats import ttest_ind
  scores=ttest_ind(df[:][target==0],df[:][target==1])[0] #Storing just the t-test scores and discarding the p-values from the result.
  
  # scores=np.argsort(scores,0)
  return [scores] if type(scores) != np.ndarray else scores

  

In [ ]:
from scipy.sparse import *
def fisher_score(X, y):
    import numpy as np
    
    from skfeature.utility.construct_W import construct_W
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature

    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)


In [ ]:

#Pearson corelation
def pearson_corr(feature,targetClass):
  import numpy as np
  coef=[np.abs(np.corrcoef(feature[i].values,targetClass)[0,1]) for i in feature.columns]
  # range(feature.shape[1])
  coef=[0 if np.isnan(i) else i for i in coef]
  return coef


In [ ]:
#gini_index 

In [ ]:
#signal to noise ratio
#using weighted one-vs-all strategy for multi-class data
def signaltonoise(feature, target, axis = 0, ddof = 0):
  import numpy as np
  classes = np.unique(target)
  if len(feature.shape)<2:
    feature = feature.reshape(-1,1)
  row, _ = feature.shape
  if len(classes) <= 2:
    m = None
    std = 0
    for each in classes:
      idx = np.where(target == each)[0]
      #convinient way of doing m1-m2
      if m is None:
        m = feature.iloc[idx, :].mean(axis)
      else:
        m -= feature.iloc[idx, :].mean(axis)

      #sd1+sd2
      std += feature.iloc[idx, :].std(axis = axis, ddof = ddof)

    return np.asanyarray(m/std)

  else:
    snr_scores = [] #for storing the weighted scores
    #using the one vs all strategy for each class with
    for each in classes:
      idx = np.where(target == each)[0]
      idxn = np.where(target != each)[0]
      m = feature.iloc[idx, :].mean(axis) - feature.iloc[idxn, :].mean(axis)
      std = feature.iloc[idx, :].std(axis = axis, ddof = ddof) + feature.iloc[idxn, :].std(axis = axis, ddof = ddof) 
      snr_scores.append((m/std) * len(idx)/row) #weighted snr

    return np.asanyarray(snr_scores).sum(axis = axis)

In [ ]:
#intentionally left blank!

In [ ]:
def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [ ]:

relief_score=reliefF(feature,target,NEIGHBOURS)

mutual_inf=mutual_info_wrapper(feature,target)

mms=MinMaxScaler()
nfeature=mms.fit_transform(feature)
chi_score,p_val=chi2(nfeature,target)

p_corr = pearson_corr(feature, target)

f_score = fisher_score(feature.values, target)

tt_score = t_test(feature, target)

snr_score = signaltonoise(feature, target)

In [ ]:
#Can skip this cell
print("Feature Scores (Unsorted)-")
print("ReliefF -",relief_score[:5])
print("Mutual Information -", mutual_inf)
print("Chi Square Test Score -",chi_score)
print("Pearson Corelation - ", p_corr)
print("Fisher Score - ", f_score)
print("T-Test Scores - ", tt_score)
print("Signal to Noise Scores - ", snr_score)

Feature Scores (Unsorted)-
ReliefF - [0.050397709381417126, 0.015370894335704561, -0.022086670820492597, -0.022715501783129123, 0.003680098330294964]
Mutual Information - [ 0.36215326  0.33373526  0.16595525 ...  0.02036548 -0.00082661
  0.06555464]
Chi Square Test Score - [4.23321274 8.00993606 2.82050613 ... 0.74141386 0.26034787 1.10973111]
Pearson Corelation -  [0.026541896784921033, 0.5405803857131632, 0.23885562398046828, 0.12039690845168322, 0.08899644437478674, 0.0010208805091560104, 0.23948542930963157, 0.0664511656541523, 0.12246062071494108, 0.08573846303703508, 0.26198780104598535, 0.1531583159711869, 0.10020599113445554, 0.128676248051197, 0.07086681885124309, 0.07202866092115452, 0.028658546919926, 0.2758875278075834, 0.0638003958441658, 0.07295220400809659, 0.1378952755154495, 0.13086275183875515, 0.041418223322166145, 0.17040186504205568, 0.09533182260498088, 0.44776682681100355, 0.24037061777161972, 0.18689599652100916, 0.4851086140576573, 0.1362529644004312, 0.2674038

In [ ]:
#The Features are sorted as per their scores
sorted_relief = feature_ranking(relief_score)[:p]
sorted_mi = feature_ranking(mutual_inf)[:p]
sorted_chi = feature_ranking(chi_score)[:p]
sorted_pc = feature_ranking(p_corr)[:p]
sorted_fs = feature_ranking(f_score)[:p]
sorted_tt = feature_ranking(tt_score)[:p]
sorted_snr = feature_ranking(snr_score)[:p]

In [ ]:
#Can Skip this Cell

print("Features after sorting -")
print("\nSorted MI -",sorted_mi)
print("\nSorted Relief -",sorted_relief)
print("\nSorted Chi -",sorted_chi)
print("\nSorted Pearson Corr -",sorted_pc)
print("\nSorted Fisher Score -",sorted_fs)
print("\nSorted T-test -",sorted_tt)
print("\nSorted SNR - ", sorted_snr)

Features after sorting -

Sorted MI - [1388  544 1002  152 1954 2143 1193 1644  186  245  845  334 1206  508
 2045 1953 2049 1661  741 1318 1707 1931 2161  950 1326  975  228 1065
  173  187 2158  835  254 1895 1979  106  335  866 2252 1612 1352  757
 1893 1705 1887  107 2198 1798 1157  367  878  909 1329 1910 1083 1054
  337  480  379  436 1883 1722 2202 1374 1488 1967 2126  936  602 1654
 1829  978  822  416  429 1999  606  243 1029  760  165 2247 1092 1605
  421 1923  255 1197 1600 1771  482 1485 1433 1872 1104  250  235 1913
  181  718 1769  589  798   66 2229 1861 1262  574 1729 1386  122  229
 1066 1633 1914 2145 2031  565  656 1158 2185    0 1195 1111  752 1820
 2257 1875 2302 1020 1109  799 1492 1294 1737 2275 1478 1578 2095   73
 2156  551 1073 1141  818 1819  234 1672  206  476 1948  168 1523  399
  596 1941 1699  432  850  468  377    1  714 1881 1755 1371  150 2116
 1089 1300 1884  364 1385  347  155 1993  308  819  997  874  782   88
 1480 1749 2021 1607  246  164 1886 134

In [ ]:
def score(a,p,target):  
  if p==1:
    return mutual_info_wrapper(pd.DataFrame(a.reshape(-1,1)),target)
    
  if p==2:    
    ndf=pd.DataFrame()
    ndf[0]=a
    reliefa=reliefF(ndf,target,NEIGHBOURS,2)
    return reliefa
  
  if p==3:    
    from sklearn.preprocessing import MinMaxScaler
    mms=MinMaxScaler() 
    a=mms.fit_transform(a.reshape(-1,1))
    chia=chi2(a,target)[0]
    return chia
  
  if p==4:
    return pearson_corr(pd.DataFrame(a.reshape(-1, 1)), target)
  
  if p==5:
    return fisher_score(a.reshape(-1,1), target)
  
  if p==6:
    return t_test(a, target)
  
  if p==7:
    return signaltonoise(pd.DataFrame(a.reshape(-1,1)), target)

In [ ]:
def get_clusters(genes,features,p,target):
  """
  genes - list of subset gene. These are the genes of picked by the score function. Please note that these are just the gene names. Their actual values are passed in the features dataframe
  features - the dataframe which contains the values of the genes
  p - this denotes the  type of score function. 1- mutual information, 2- reliefF, 3- chi square test.
  target - target is a pandas series of target clases for each observation
  """
  clusters={}
  cluster_gene={}
  x,y=0,0
  genes_copy_1=np.copy(genes)
  while(len(genes_copy_1)>0):
    # print("Starting New Iteration with", len(genes_copy_1),"number of genes!")
    genes_copy_2=np.copy(genes_copy_1)
    r_gene=genes_copy_2[0]
    r_gene_values=features[r_gene].values

    clusters[str(r_gene)]=[]
    
    genes_copy_2=np.delete(genes_copy_2,0)
    genes_copy_1=np.delete(genes_copy_1,0)
    
    
    
    r_score=score(r_gene_values,p,target)[0]
    
    # print("\nCluster number=",len(clusters))
    # print("First feature =========================j1=",r_gene,"\n")
    x+=1
    # print("Intial Relevance Score",r_score)

    while(len(genes_copy_2)>0):
      
      gs=genes_copy_2[0]
      gene=features[gs].values

      y+=1      
      
      a_plus=np.add(r_gene_values,gene,dtype='float64') #creating A+
      a_minus=np.subtract(r_gene_values,gene,dtype='float64') #Creating A-

      a_plus_score=score(a_plus,p,target)[0]
      a_minus_score=score(a_minus,p,target)[0]
      
      new_score=a_plus_score if a_plus_score>a_minus_score else a_minus_score
      # print("Gene",gs,"+ Score",a_plus_score,"- Score",a_minus_score)

      if new_score>r_score:

        if a_plus_score==new_score:

          # print("Gene Under Consideration",gs)
          # print("Initial Relevance",r_score,"Final Relevance",a_plus_score,r_score<a_plus_score)

          clusters[str(r_gene)].append(str(gs)+"+")
          r_gene_values=a_plus[:]
          r_score=a_plus_score

          # print("cluster member = +",gs,"\tRelevance Changed to",r_score)

        elif a_minus_score==new_score:

          # print("Gene Under Consideration",gs)
          # print("Initial Relevance",r_score,"Final Relevance",a_minus_score,r_score<a_minus_score)
          
          clusters[str(r_gene)].append(str(gs)+"-")
          r_gene_values=a_minus[:]
          r_score=a_minus_score

        #   print("cluster member = -",gs,"\tRelevance Changed to",r_score)
        # print("Gene",gs,"selected!",np.where(genes_copy_1 == gs))
        genes_copy_1 = np.delete(genes_copy_1, np.where(genes_copy_1 == gs))      
      genes_copy_2=np.delete(genes_copy_2,0)
    
    # for each in clusters[str(r_gene)]:
    #     genes_copy_1=np.delete(genes_copy_1,np.where(genes_copy_1==each))
    cluster_gene[r_gene]=r_gene_values

  #   print("\nFinal Relevance Score",r_score)
  print("Clusters formed! Returning Clusters and Gene Representatives")
  return clusters,cluster_gene

In [ ]:
#Please skip this cell in favour of the next if you are using a local runtime instead of colab.
"""
Computes the clusters via parallel processing

"""
"""
import time
start=time.time()
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
(mi_cluster,gene_repre_1),(relief_cluster,gene_repre_2),(chi_cluster,gene_repre_3)=[pool.apply(getClusters, args=x) for x in [(sorted_mi,feature,1,target),(sorted_relief,feature,2,target),(sorted_chi,feature,3,target)]]
pool.close()
end=time.time()
print(end-start)
"""

'\nimport time\nstart=time.time()\nimport multiprocessing as mp\npool = mp.Pool(mp.cpu_count())\n(mi_cluster,gene_repre_1),(relief_cluster,gene_repre_2),(chi_cluster,gene_repre_3)=[pool.apply(getClusters, args=x) for x in [(sorted_mi,feature,1,target),(sorted_relief,feature,2,target),(sorted_chi,feature,3,target)]]\npool.close()\nend=time.time()\nprint(end-start)\n'

In [ ]:
mi_cluster, gene_repre_1 = get_clusters(sorted_mi, feature, 1, target)
relief_cluster ,gene_repre_2 = get_clusters(sorted_relief, feature, 2, target)
chi_cluster, gene_repre_3 = get_clusters(sorted_chi, feature, 3, target)

Clusters formed! Returning Clusters and Gene Representatives
Clusters formed! Returning Clusters and Gene Representatives
Clusters formed! Returning Clusters and Gene Representatives


In [ ]:

pc_cluster, gene_repre_4 = get_clusters(sorted_pc, feature, 4, target)
fs_cluster, gene_repre_5 = get_clusters(sorted_fs, feature, 5, target)
tt_cluster, gene_repre_6 = get_clusters(sorted_tt, feature, 6, target)

Clusters formed! Returning Clusters and Gene Representatives
Clusters formed! Returning Clusters and Gene Representatives
Clusters formed! Returning Clusters and Gene Representatives


In [ ]:
snr_cluster, gene_repre_7 = get_clusters(sorted_snr, feature, 7, target)

Clusters formed! Returning Clusters and Gene Representatives


In [ ]:
print("Number of MI Clusters formed -",len(mi_cluster))
print("Number of ReliefF Clusters formed -",len(relief_cluster))
print("Number of ChiSq. Clusters formed -",len(chi_cluster))
print("Number of Pearson Clusters formed -",len(pc_cluster))
print("Number of Fisher Score Clusters formed -",len(fs_cluster))
print("Number of T-Test Clusters formed -",len(tt_cluster))
print("Number of SNR Clusters formed -", len(snr_cluster))

Number of MI Clusters formed - 54
Number of ReliefF Clusters formed - 12
Number of ChiSq. Clusters formed - 39
Number of Pearson Clusters formed - 7
Number of Fisher Score Clusters formed - 9
Number of T-Test Clusters formed - 12
Number of SNR Clusters formed - 14


In [ ]:
gene_repre_1 = pd.DataFrame(gene_repre_1)
gene_repre_2 = pd.DataFrame(gene_repre_2)
gene_repre_3 = pd.DataFrame(gene_repre_3)
gene_repre_4 = pd.DataFrame(gene_repre_4)
gene_repre_5 = pd.DataFrame(gene_repre_5)
gene_repre_6 = pd.DataFrame(gene_repre_6)
gene_repre_7 = pd.DataFrame(gene_repre_7)

In [ ]:
print("MI -",gene_repre_1.head())
print("ReliefF -",gene_repre_2.head())
print("Chi -",gene_repre_3.head())
print("Pearson -",gene_repre_4.head())
print("Fisher Score -",gene_repre_5.head())
print("T-Test -",gene_repre_6.head())
print("SNR -",gene_repre_7.head())

MI -       1388     1193    334      1318  ...    23      1573    477     354 
0  10.1886 -10.3985 -2.8721  12.5458  ...  7.5323  3.7401  1.3653  3.0216
1   9.4296  -7.8566 -2.6802  13.4258  ...  8.5748  2.3794  2.0892  2.9033
2   6.8954  -8.2212 -1.4287  10.4759  ...  9.8648  3.0841  1.6290  2.8229
3  10.3398  -4.7042 -1.8044   9.4123  ...  6.7251  4.5457  1.8211  6.5289
4   9.7931  -8.8772 -0.6759  10.0126  ...  7.1531  1.7820  1.0683  2.7075

[5 rows x 54 columns]
ReliefF -      63       1953     2289     565   ...     1467     286      1780    1726
0  7.3056  10.3002   8.9733   1.5153  ...  -9.7188   5.6944   3.7661  0.3419
1  6.5486  11.6163  12.3314   0.7962  ...  -6.9983   5.9472   5.4671  0.2437
2  6.9280   8.6558  16.8619   7.6460  ...  -9.7865   5.8401   4.2402  0.3628
3  4.3371  20.7725  19.2294   2.0250  ... -11.0355   5.4688  12.3256  0.9463
4  6.3834  16.6092  12.6048  12.1985  ...  -3.5591  15.0916  14.1811  2.0023

[5 rows x 12 columns]
Chi -       1388    1954    782  

In [ ]:
gene_repre_1.to_csv("%s_p%sRepresentative_Genes_1.csv"%(DATASET, p),index=False)
gene_repre_2.to_csv("%s_p%sRepresentative_Genes_2.csv"%(DATASET, p),index=False)
gene_repre_3.to_csv("%s_p%sRepresentative_Genes_3.csv"%(DATASET, p),index=False)
gene_repre_4.to_csv("%s_p%sRepresentative_Genes_4.csv"%(DATASET, p),index=False)
gene_repre_5.to_csv("%s_p%sRepresentative_Genes_5.csv"%(DATASET, p),index=False)
gene_repre_6.to_csv("%s_p%sRepresentative_Genes_6.csv"%(DATASET, p),index=False)
gene_repre_7.to_csv("%s_p%sRepresentative_Genes_7.csv"%(DATASET, p),index=False)

In [ ]:
"""
Saving the clusters to JSON files, this preserves the gene selection sequence
"""
with open('%s_p%smi_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(mi_cluster, fp)

with open('%s_p%srelief_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(relief_cluster, fp)


with open('%s_p%schi_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(chi_cluster, fp)

with open('%s_p%spc_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(pc_cluster, fp)

with open('%s_p%sfs_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(fs_cluster, fp)

with open('%s_p%stt_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(tt_cluster, fp)

with open('%s_p%ssnr_cluster.json'%(DATASET, p), 'w') as fp:
    json.dump(snr_cluster, fp)

In [ ]:
def sort_keys(scores,gene_repre,target,flag=True):
  score_dict={}
  x=0
  for i in gene_repre.columns:
    score_dict[i]=scores[x]
    x+=1
  return [k for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)]

In [ ]:
"""
feature_ranking cannot be used here because it sorts and returns the indices from 0-1
They need to be sorted using a different function
"""
sorted_mi_keys=sort_keys(mutual_info_wrapper(gene_repre_1,target),gene_repre_1,target,True)

sorted_relief_keys=sort_keys(reliefF(gene_repre_2,target,k=NEIGHBOURS,repetitions=5),gene_repre_2,target,True)

mms=MinMaxScaler()
nfeature=mms.fit_transform(gene_repre_3)
chi_score,p_val=chi2(nfeature,target)
sorted_chi_keys=sort_keys(chi_score,gene_repre_3,target,False)

sorted_pc_keys=sort_keys(pearson_corr(gene_repre_4,target),gene_repre_4,target,True)

sorted_fs_keys=sort_keys(fisher_score(gene_repre_5.values,target),gene_repre_5,target,True)

sorted_tt_keys=sort_keys(t_test(gene_repre_6,target),gene_repre_6,target,True)

sorted_snr_keys = sort_keys(signaltonoise(gene_repre_7, target), gene_repre_7, target, True)

In [ ]:
print("MI cluster after sorting - ",sorted_mi_keys)
print("Relief cluster after sorting - ",sorted_relief_keys)
print("Chi cluster after sorting - ",sorted_chi_keys)
print("Pearson cluster after sorting - ",sorted_pc_keys)
print("Fisher cluster after sorting - ",sorted_fs_keys)
print("T-Test cluster after sorting - ",sorted_tt_keys)
print("SNR cluster after sorting - ",sorted_snr_keys)

MI cluster after sorting -  [1318, 1352, 1388, 1193, 1699, 2202, 1371, 950, 1798, 997, 760, 2229, 1083, 1065, 1620, 1750, 1820, 1859, 334, 1856, 878, 644, 1370, 572, 26, 1281, 1020, 611, 936, 1833, 1656, 799, 981, 921, 2165, 797, 1001, 668, 1685, 129, 1285, 1749, 1272, 1159, 1832, 1647, 70, 23, 184, 956, 1112, 477, 1573, 354]
Relief cluster after sorting -  [2289, 63, 1953, 1985, 347, 1707, 565, 1833, 286, 1467, 1780, 1726]
Chi cluster after sorting -  [782, 845, 254, 1388, 1914, 741, 1954, 1066, 1895, 152, 544, 1707, 565, 936, 2049, 1065, 1979, 1002, 2021, 1021, 950, 335, 347, 570, 150, 969, 671, 871, 535, 1820, 155, 1685, 850, 265, 293, 83, 613, 2134, 1906]
Pearson cluster after sorting -  [1388, 544, 1707, 429, 47, 1573, 1833]
Fisher cluster after sorting -  [1954, 122, 909, 975, 508, 900, 637, 500, 535]
T-Test cluster after sorting -  [1388, 245, 83, 1644, 565, 2137, 1352, 145, 2219, 1840, 830, 1833]
SNR cluster after sorting -  [1388, 831, 1954, 1073, 450, 720, 1539, 1619, 76, 519

In [ ]:
LOOCV=LeaveOneOut()
data_KNN=KNeighborsClassifier(n_neighbors= int(feature.shape[0] ** 0.5))
data_SVM=SVC(kernel='rbf',gamma='scale')
data_NB=GaussianNB()
data_Tree= DecisionTreeClassifier()
rows=feature.shape[0]
classifiers=["NB","KNN","Tree","SVM"]

keys_list=[sorted_mi_keys, sorted_relief_keys, sorted_chi_keys, sorted_pc_keys, 
           sorted_fs_keys, sorted_tt_keys, sorted_snr_keys]
cluster_list=[gene_repre_1, gene_repre_2, gene_repre_3, gene_repre_4, 
              gene_repre_5, gene_repre_6, gene_repre_7]

In [ ]:
acc_matrix = pd.DataFrame()
for i in range(1,6):
  """
  Make a dataframe out of i keys from each gene_representatives from their 
  respective sorted keys. 
  Than use LOOCV to measure accuracy on Train Dataset.
  """
  cluster_df=pd.DataFrame()
  s=0
  for x in range(len(cluster_list)):
    tem_df = cluster_list[x][keys_list[x][:i]]
    for x in range(i):
      cluster_df[s] = tem_df.iloc[:,x]
      s+=1
  print(cluster_df.shape)
  acc=0
  individual_acc = np.zeros(4)
  
  for train_index,test_index in LOOCV.split(cluster_df):
    """
    Data is divided into train-test splits and then polling method is used 
    to find the classification results (ensemble of KNN,SVM,NB,Decision Tree)
    """
    train_data,train_labels=cluster_df.iloc[train_index,:],target[train_index]
    test_data,test_labels=cluster_df.iloc[test_index,:],target[test_index].values.tolist()[0]
    data_KNN.fit(train_data,train_labels)
    data_SVM.fit(train_data,train_labels)
    data_NB.fit(train_data,train_labels)
    data_Tree.fit(train_data,train_labels)

    class_list = [data_NB, data_KNN, data_Tree, data_SVM]
    results=[]
    for x in range(4):
      tem_result = class_list[x].predict(test_data)[0]
      if tem_result == test_labels:
        individual_acc[x]+=1
      results.append(tem_result)
    polling_result=0
    max_freq=0
    for x in results:
      freq=results.count(x)
      if freq>max_freq:
        max_freq=freq
        polling_result=x
    if polling_result == test_labels:
      acc+=1
  individual_acc = np.round(individual_acc/cluster_df.shape[0],4)
  individual_acc = np.append(individual_acc, np.round(acc/cluster_df.shape[0],4))
  print(individual_acc)
  acc_matrix[i] = individual_acc
acc_matrix = acc_matrix.T
acc_matrix.columns = classifiers[:]+['Ensemble']

(63, 7)
[1. 1. 1. 1. 1.]
(63, 14)
[1.     1.     0.9841 1.     1.    ]
(63, 21)
[1.     1.     0.9683 1.     1.    ]
(63, 28)
[1. 1. 1. 1. 1.]
(63, 35)
[1.     1.     0.9841 1.     1.    ]


In [ ]:
print(acc_matrix)
acc_matrix.to_csv("%s_p%sAccuracy_Matrix.csv"%(DATASET, p))

    NB  KNN    Tree  SVM  Ensemble
1  1.0  1.0  1.0000  1.0       1.0
2  1.0  1.0  0.9841  1.0       1.0
3  1.0  1.0  0.9683  1.0       1.0
4  1.0  1.0  1.0000  1.0       1.0
5  1.0  1.0  0.9841  1.0       1.0


In [ ]:
"""
Downloads all the Files generated for this dataset. Please note: The dataset name must be specified in the DATASET 
variable before proceeding with the steps below.

Incase of 'failed to fetch' error, please rerun thiis cell.
"""
for x in range(1,4):
  files.download("%s_p%sRepresentative_Genes_%d.csv"%(DATASET, p, x))

for x in ['mi','chi','relief']:
  files.download("%s_p%s%s_cluster.json"%(DATASET, p, x))

files.download("%s_p%sAccuracy_Matrix.csv"%(DATASET, p))

In [ ]:
#*****************-----Notebook Training Code Ends Here. Below are cells for loading pre-calculated values for Representative Genes and Cluster Sequences.-------*****************************

In [ ]:
#Cell Left Empty Intentionally.

In [ ]:
"""
Loading the JSON files from memory
"""
with open("%s_mi_cluster.json"%(DATASET),"r") as fp:
  mi_cluster=json.load(fp)
with open("%s_chi_cluster.json"%(DATASET),"r") as fp:
  chi_cluster=json.load(fp)
with open("%s_relief_cluster.json"%(DATASET),"r") as fp:
  relief_cluster=json.load(fp)

In [ ]:
"""
Loading Representative Genes from Memory
"""
gene_repre_1 = pd.read_csv("%s_Representative_Genes_1.csv"%(DATASET),index_col = None)
gene_repre_2 = pd.read_csv("%s_Representative_Genes_2.csv"%(DATASET),index_col = None)
gene_repre_3 = pd.read_csv("%s_Representative_Genes_3.csv"%(DATASET),index_col = None)

In [ ]:
data_Tree.get_params()